In [11]:
from core.db import get_engine
from core.libs import pd, text, np

engine = get_engine()

💻 Conectado a la base de datos helloworldtree


In [12]:
#First, we are going to extract the last register of survival from main database that is in:

path = r"C:\Users\HeyCe\OneDrive - World Tree Technologies Inc\NULL INSPECTION with maindatabase.xlsx"
sheet = 'maindatabase_extract'

luka = pd.read_excel(path, sheet_name=sheet)

print(len(luka))
print(luka.head(0))

423
Empty DataFrame
Columns: [Contract Code, 2024 Survival% Inventory + Estimate, 2024 #Trees Inventory + Estimate, Filter 1: zeros, Filter: Discrepancies with Estimate]
Index: []


In [13]:
path = r"C:\Users\HeyCe\OneDrive - World Tree Technologies Inc\NULL INSPECTION with maindatabase.xlsx"
sheet = 'maindatabase x case'

contract_codes_and_cases = pd.read_excel(path, sheet_name=sheet)

print(contract_codes_and_cases.head(0))

Empty DataFrame
Columns: [contract_code, null_survival_case, Search]
Index: []


In [14]:
# JOIN CTI → IMC
join_query = text("""
    SELECT
        cti.contract_code,
        cti.status,
        cti.planting_year,
        cti.etp_year,
        cti.region,
        (2025 - cti.planting_year) AS age,
        imc.inventory_year,
        imc.survival
    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.inventory_metrics_current imc
        ON cti.contract_code = imc.contract_code;
""")

cti_imc = pd.read_sql(join_query, engine)

cti_imc.head()


,contract_code,status,planting_year,etp_year,region,age,inventory_year,survival
0,MX0071,Active,2024,2023,Mexico,1,NaN,1.0000
1,MX0099,Pending POD,2025,2024,None,0,NaN,NaN
2,US0164,Active,2024,2023,USA,1,2025.0,0.5789
3,US0166,Active,2025,2024,None,0,NaN,1.0000
4,US0168,Active,2025,2024,None,0,NaN,1.0000


In [15]:
dfs = [luka, contract_codes_and_cases, cti_imc]

for df in dfs:
    # obtener nombre
    name = [n for n in globals() if globals()[n] is df][0]
    print(f"\n=== {name} ===")

    # imprimir encabezados
    print(f"Total de registros: {len(df)}")
    print(list(df.columns))



=== luka ===
Total de registros: 423
['Contract Code', '2024 Survival% Inventory + Estimate', '2024 #Trees Inventory + Estimate', 'Filter 1: zeros', 'Filter: Discrepancies with Estimate']

=== contract_codes_and_cases ===
Total de registros: 244
['contract_code', 'null_survival_case', 'Search']

=== cti_imc ===
Total de registros: 462
['contract_code', 'status', 'planting_year', 'etp_year', 'region', 'age', 'inventory_year', 'survival']


In [16]:
luka_ren = luka.rename(columns={
    "Contract Code": "contract_code",
    "2024 Survival% Inventory + Estimate": "luka_survival_2024",
    "2024 #Trees Inventory + Estimate": "luka_trees_2024",
    "Filter 1: zeros": "luka_filter_zeros",
    "Filter: Discrepancies with Estimate": "luka_filter_discrepancies"
})


In [17]:
df_final = (
    cti_imc
    .merge(
        contract_codes_and_cases[
            ["contract_code", "null_survival_case", "Search"]
        ],
        on="contract_code",
        how="left"
    )
    .merge(
        luka_ren[
            [
                "contract_code",
                "luka_survival_2024",
                "luka_trees_2024",
                "luka_filter_zeros",
                "luka_filter_discrepancies",
            ]
        ],
        on="contract_code",
        how="left"
    )
)


In [18]:
df_final["ultimate_survival"] = np.nan
df_final["new_inventory_type"] = np.nan


In [19]:
# ============================================
# CASE 2: Trees minor 3 years old
# ============================================

mask_case2 = df_final["null_survival_case"] == "CASE 2: Trees minor 3 years old"

# --------------------------------------------
# Regla 1:
#   when:
#       null_survival_case == CASE 2
#       survival == luka_survival_2024 (ambos no nulos)
#   then:
#       ultimate_survival = survival
#       new_inventory_type = "maindatabase last record"
# --------------------------------------------
mask_rule1 = (
    mask_case2
    & df_final["survival"].notna()
    & df_final["luka_survival_2024"].notna()
    & (df_final["survival"] == df_final["luka_survival_2024"])
)

df_final.loc[mask_rule1, "ultimate_survival"] = df_final.loc[mask_rule1, "survival"]
df_final.loc[mask_rule1, "new_inventory_type"] = "maindatabase last record"


C:\Users\HeyCe\AppData\Local\Temp\ipykernel_8484\2609590857.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'maindatabase last record' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[mask_rule1, "new_inventory_type"] = "maindatabase last record"


In [20]:
# --------------------------------------------
# Regla 2:
#   when:
#       null_survival_case == CASE 2
#       ultimate_survival IS NULL
#       new_inventory_type IS NULL
#       (
#         (survival is NaN AND luka_survival_2024 NOT NaN)
#          OR
#         (survival != luka_survival_2024 AND ambos NOT NaN)
#       )
#   then:
#       ultimate_survival = luka_survival_2024
#       new_inventory_type = "maindatabase last record"
# --------------------------------------------

mask_base = (
    mask_case2
    & df_final["ultimate_survival"].isna()
    & df_final["new_inventory_type"].isna()
)

cond_survival_nan_luka_notnan = (
    df_final["survival"].isna()
    & df_final["luka_survival_2024"].notna()
)

cond_survival_diff_both_notnan = (
    df_final["survival"].notna()
    & df_final["luka_survival_2024"].notna()
    & (df_final["survival"] != df_final["luka_survival_2024"])
)

mask_rule2 = mask_base & (cond_survival_nan_luka_notnan | cond_survival_diff_both_notnan)

df_final.loc[mask_rule2, "ultimate_survival"] = df_final.loc[mask_rule2, "luka_survival_2024"]
df_final.loc[mask_rule2, "new_inventory_type"] = "maindatabase last record"


In [21]:
# ============================================
# CASE 2 - base
# ============================================
mask_case2 = df_final["null_survival_case"] == "CASE 2: Trees minor 3 years old"

# status que contienen 'Pending' (ej. 'Pending POD', 'Pending Planting', etc.)
mask_pending = df_final["status"].fillna("").str.contains("Pending", case=False)

# --------------------------------------------
# REGLA PENDING (CASE 2 + Pending)
#   → No tocar ultimate_survival (se queda NaN)
#   → new_inventory_type = "Not planted contract"
# --------------------------------------------
mask_case2_pending = (
    mask_case2
    & mask_pending
    & df_final["new_inventory_type"].isna()
)

df_final.loc[mask_case2_pending, "new_inventory_type"] = "Not planted contract"

# --------------------------------------------
# REGLA 3 (ELSE) - CASE 2, NO Pending
#   → ultimate_survival = 1
#   → new_inventory_type = "Not planted contract"
# --------------------------------------------
mask_rule3 = (
    mask_case2
    & ~mask_pending                       # 👈 excluir Pending
    & df_final["ultimate_survival"].isna()
    & df_final["new_inventory_type"].isna()
)

df_final.loc[mask_rule3, "ultimate_survival"] = 1.0
df_final.loc[mask_rule3, "new_inventory_type"] = "Not planted contract"


In [22]:
df_final.loc[mask_case2, [
    "contract_code",
    "status",
    "null_survival_case",
    "survival",
    "luka_survival_2024",
    "ultimate_survival",
    "new_inventory_type"
]].head(20)


,contract_code,status,null_survival_case,survival,luka_survival_2024,ultimate_survival,new_inventory_type
0,MX0071,Active,CASE 2: Trees minor 3 years old,1.0,0.666667,0.666667,maindatabase last record
1,MX0099,Pending POD,CASE 2: Trees minor 3 years old,NaN,NaN,NaN,Not planted contract
3,US0166,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
4,US0168,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
5,CR0112,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
6,US0169,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
7,US0170,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
8,US0167,Active,CASE 2: Trees minor 3 years old,1.0,NaN,1.000000,Not planted contract
9,MX0123,Pending POD,CASE 2: Trees minor 3 years old,NaN,NaN,NaN,Not planted contract
49,MX0100,Pending POD,CASE 2: Trees minor 3 years old,NaN,NaN,NaN,Not planted contract


In [23]:
mask_fix_pending_case2 = (
    (df_final["null_survival_case"] == "CASE 2: Trees minor 3 years old")
    & df_final["status"].str.contains("Pending", na=False)
    & (df_final["new_inventory_type"] == "new contract default survival")
)

df_final.loc[mask_fix_pending_case2, "ultimate_survival"] = np.nan
df_final.loc[mask_fix_pending_case2, "new_inventory_type"] = None


In [24]:
# ============================================
# CASE 5 - Status really, really close to OOT
# ============================================

case5_label = "Case 5: Status really, really close to OOT"

mask_case5 = (
    df_final["null_survival_case"]
    .fillna("")
    .str.strip()
    .str.lower()
    == case5_label.lower()
)

mask_rule_case5 = (
    mask_case5
    & df_final["ultimate_survival"].isna()
    & df_final["new_inventory_type"].isna()
)

df_final.loc[mask_rule_case5, "ultimate_survival"] = df_final.loc[mask_rule_case5, "luka_survival_2024"]
df_final.loc[mask_rule_case5, "new_inventory_type"] = "maindatabase last record"


In [25]:
df_final.loc[mask_case5, [
    "contract_code", "status",
    "null_survival_case",
    "survival", "luka_survival_2024",
    "ultimate_survival", "new_inventory_type"
]]


,contract_code,status,null_survival_case,survival,luka_survival_2024,ultimate_survival,new_inventory_type
44,MX0005,RFT,"Case 5: Status really, really close to OOT",NaN,0.95,0.95,maindatabase last record
137,CR0045,RFT,"Case 5: Status really, really close to OOT",NaN,0.83,0.83,maindatabase last record
161,US0001,RFT,"Case 5: Status really, really close to OOT",NaN,0.00,0.00,maindatabase last record
401,CR0072,Rescinded,"Case 5: Status really, really close to OOT",NaN,0.00,0.00,maindatabase last record


In [26]:
# ============================================
# CASE 1 - Out of Program (VERSIÓN CORREGIDA)
# ============================================

case1_label = "CASE 1: Out of Program"

mask_case1 = (
    df_final["null_survival_case"]
    .fillna("")
    .str.strip()
    .str.lower()
    == case1_label.lower()
)

# Solo asignar si aún no se asignó nada
mask_case1_unassigned = (
    mask_case1
    & df_final["new_inventory_type"].isna()
    & df_final["ultimate_survival"].isna()
)

# --------------------------------------------
# Condición 1: Luka survival disponible y != 0
# --------------------------------------------

mask_case1_luka_valid = (
    mask_case1_unassigned
    & df_final["luka_survival_2024"].notna()
    & (df_final["luka_survival_2024"] != 0)
)

df_final.loc[mask_case1_luka_valid, "ultimate_survival"] = df_final.loc[mask_case1_luka_valid, "luka_survival_2024"]
df_final.loc[mask_case1_luka_valid, "new_inventory_type"] = "out of program"

# --------------------------------------------
# Condición 2: else (luka survival NULL o == 0)
# --------------------------------------------

mask_case1_else = (
    mask_case1_unassigned
    & (
        df_final["luka_survival_2024"].isna()
        | (df_final["luka_survival_2024"] == 0)
    )
)

df_final.loc[mask_case1_else, "ultimate_survival"] = np.nan
df_final.loc[mask_case1_else, "new_inventory_type"] = "out of program"


In [27]:
df_final.loc[mask_case1, [
    "contract_code", "status", "null_survival_case",
    "survival", "luka_survival_2024",
    "ultimate_survival", "new_inventory_type"
]]


,contract_code,status,null_survival_case,survival,luka_survival_2024,ultimate_survival,new_inventory_type
17,CR0031,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
18,CR0032,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
19,CR0023,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
20,CR0024,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
21,CR0025,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
22,CR0026,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
23,CR0027,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
25,US0027,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
26,US0021,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program
28,CR0014,Out of Program,CASE 1: Out of Program,NaN,0.00,NaN,out of program


In [28]:
df_final = df_final[df_final["null_survival_case"].notna()].copy()


In [29]:
# ============================================
# CASE 4 - Regla universal
# ============================================

mask_case4 = df_final["null_survival_case"] == "CASE 4: Other"

mask_case4_unassigned = (
    mask_case4
    & df_final["ultimate_survival"].isna()
    & df_final["new_inventory_type"].isna()
)

df_final.loc[mask_case4_unassigned, "ultimate_survival"] = df_final.loc[mask_case4_unassigned, "luka_survival_2024"]
df_final.loc[mask_case4_unassigned, "new_inventory_type"] = "maindatabase last record"


In [30]:
result = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\survival_nulls_classified.xlsx"

df_final.to_excel(result, index=False)

In [31]:
import os

os.startfile(r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\survival_nulls_classified.xlsx")


In [32]:

q_cols = text("""
SELECT
    column_name,
    data_type,
    is_nullable,
    character_maximum_length,
    column_default
FROM information_schema.columns
WHERE table_schema = 'masterdatabase'
  AND table_name = 'inventory_metrics'
ORDER BY ordinal_position;
""")

df_cols = pd.read_sql(q_cols, engine)
df_cols




,column_name,data_type,is_nullable,character_maximum_length,column_default
0,rel_path,text,YES,None,None
1,contract_code,text,YES,None,None
2,tree_age,text,YES,None,None
3,inventory_year,bigint,YES,None,None
4,inventory_date,date,YES,None,None
5,tht_mean,double precision,YES,None,None
6,tht_std,double precision,YES,None,None
7,mht_mean,double precision,YES,None,None
8,mht_std,double precision,YES,None,None
9,dbh_mean,double precision,YES,None,None


In [33]:
q_fk = text("""
SELECT
    tc.constraint_name,
    tc.table_name AS source_table,
    kcu.column_name AS source_column,
    ccu.table_name AS target_table,
    ccu.column_name AS target_column
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
     ON tc.constraint_name = kcu.constraint_name
JOIN information_schema.constraint_column_usage AS ccu
     ON ccu.constraint_name = tc.constraint_name
WHERE tc.table_schema = 'masterdatabase'
  AND tc.table_name = 'inventory_metrics'
  AND tc.constraint_type = 'FOREIGN KEY';
""")

pd.read_sql(q_fk, engine)


,constraint_name,source_table,source_column,target_table,target_column
0,im_cti_fk,inventory_metrics,contract_code,contract_tree_information,contract_code


## Ahora dejaremos en la base de datos principal los registros de maindatabase de survival pero separados

#Primero reimportamos la tabla Luka es decir SMA o los registros de survival de maindatabase extraidos de Excel porque estoy bien güey

In [64]:
#First, we are going to extract the last register of survival from main database that is in:

path = r"C:\Users\HeyCe\OneDrive - World Tree Technologies Inc\NULL INSPECTION with maindatabase.xlsx"
sheet = 'maindatabase_extract'

luka = pd.read_excel(path, sheet_name=sheet)

print(len(luka))
print(luka.head(0))

423
Empty DataFrame
Columns: [Contract Code, 2024 Survival% Inventory + Estimate, 2024 #Trees Inventory + Estimate, Filter 1: zeros, Filter: Discrepancies with Estimate]
Index: []


In [68]:
df_luka_archive = luka_ren[[
    "contract_code",
    "luka_survival_2024",
    "luka_trees_2024"
]].copy()

df_luka_archive = df_luka_archive.rename(columns={
    "luka_survival_2024": "survival_value",
    "luka_trees_2024": "trees_value"
})

df_luka_archive["source_label"] = "LUKA_2024"
df_luka_archive["source_year"]  = 2024
df_luka_archive["notes"]        = "Full Luka survival + absolute trees"

#====================================================
insert_query = text("""
    INSERT INTO masterdatabase.survival_maindatabase_archive
        (contract_code, survival_value, trees_value, source_label, source_year, notes)
    VALUES (:contract_code, :survival_value, :trees_value, :source_label, :source_year, :notes)
""")

with engine.begin() as conn:
    conn.execute(insert_query, df_luka_archive.to_dict(orient="records"))


#Ahora sí a calcular diferencias entre IMC y SMA

In [70]:
luka_ren = luka.rename(columns={
    "Contract Code": "contract_code",
    "2024 Survival% Inventory + Estimate": "luka_survival_2024",
    "2024 #Trees Inventory + Estimate": "luka_trees_2024",
    "Filter 1: zeros": "luka_filter_zeros",
    "Filter: Discrepancies with Estimate": "luka_filter_discrepancies"
})


In [82]:
# ======================================
# 1. Cargar IMC (462)
# ======================================

q_imc = """
    SELECT
        contract_code,
        survival AS imc_survival,
        total_trees AS imc_total_trees,
        inventory_year
    FROM masterdatabase.inventory_metrics_current
"""


df_imc = pd.read_sql(q_imc, engine)


# ======================================
# 2. Cargar SMA (Luka 2024) - survival + trees
# ======================================

q_sma = """
    SELECT
        contract_code,
        survival_value AS sma_survival,
        trees_value AS sma_trees
    FROM masterdatabase.survival_maindatabase_archive
    WHERE source_label = 'LUKA_2024'
"""

df_sma = pd.read_sql(q_sma, engine)


# ======================================
# 3. Cargar CTI (status, planting year, region)
# ======================================

q_cti = """
    SELECT
        contract_code,
        status,
        planting_year,
        etp_year,
        region
    FROM masterdatabase.contract_tree_information
"""

df_cti = pd.read_sql(q_cti, engine)


# ======================================
# 4. MERGE FINAL
# Universo = IMC  (462 filas)
# ======================================

df = (
    df_imc
    .merge(df_sma, on="contract_code", how="left")
    .merge(df_cti, on="contract_code", how="left")
)

df["diff_survival"] = df["imc_survival"] - df["sma_survival"]


# ======================================
# 5. Asignación de CASES (1 a 5)
# ======================================

def assign_case(row):

    # CASE 1 — Out of Program
    if row["status"] == "Out of Program":
        return "CASE 1: Out of Program"

    # CASE 2 — Edad < 3 años
    age = 2025 - (row["planting_year"] or 9999)
    if age < 3:
        return "CASE 2: Trees < 3 years old"

    # CASE 3 — Pending / No status válido
    if row["status"] is None or str(row["status"]).startswith("Pending"):
        return "CASE 3: Pending"

    # CASE 4 — SMA disponible y discrepancia
    if pd.notna(row["sma_survival"]) and row["diff_survival"] != 0:
        return "CASE 4: SMA discrepancy"

    # CASE 5 — Todo coincide
    if pd.notna(row["sma_survival"]) and row["diff_survival"] == 0:
        return "CASE 5: Match"

    # Default — IMC sin SMA
    return "NO SMA record"


df["null_survival_case"] = df.apply(assign_case, axis=1)


# ======================================
# 6. Revisión rápida de conteos
# ======================================

print("Total IMC:", len(df))                      # esperado: 462
print("Con SMA:", df["sma_survival"].notna().sum())
print("Sin SMA:", df["sma_survival"].isna().sum())
print(df["null_survival_case"].value_counts())


# ======================================
# 7. Export opcional (para Excel)
# ======================================

# df.to_excel("reconciliation_full.xlsx", index=False)


Total IMC: 462
Con SMA: 422
Sin SMA: 40
null_survival_case
CASE 4: SMA discrepancy        175
CASE 1: Out of Program         139
CASE 2: Trees < 3 years old    137
CASE 5: Match                   11
Name: count, dtype: int64


In [73]:
df_compare = df_imc.merge(
    df_sma,
    on="contract_code",
    how="left"
)
df_compare["diff"] = df_compare["imc_survival"] - df_compare["sma_survival"]

print("len(df_compare) =", len(df_compare))  # <-- debe ser 423 SIEMPRE


len(df_compare) = 462


In [74]:
q_cti = """
    SELECT
        contract_code,
        status,
        planting_year,
        etp_year,
        region
    FROM masterdatabase.contract_tree_information
"""

df_cti = pd.read_sql(q_cti, engine)

df_compare = df_compare.merge(
    df_cti,
    on="contract_code",
    how="left"
)

print(len(df_compare))  # <-- debe ser 423 SIEMPRE

462


In [75]:
has_imc = df_compare["imc_survival"].notna()

n_total = len(df_compare)
n_with_imc = has_imc.sum()
n_without_imc = (~has_imc).sum()

print("Total SMA (Luka):", n_total)
print("Con IMC:", n_with_imc)
print("Sin IMC:", n_without_imc)


Total SMA (Luka): 462
Con IMC: 291
Sin IMC: 171


In [77]:
mask_equal = has_imc & (df_compare["imc_survival"] == df_compare["sma_survival"])
mask_diff  = has_imc & (df_compare["imc_survival"] != df_compare["sma_survival"])

n_equal = mask_equal.sum()
n_diff  = mask_diff.sum()

print("Iguales:", n_equal)      # deberías volver a ver tus 71
print("Diferentes:", n_diff)    # y tus 214
print("Check:", n_equal + n_diff, "vs", n_with_imc)  # debe cuadrar


Iguales: 71
Diferentes: 220
Check: 291 vs 291


In [78]:
imc_null_sma_value = df_compare[
    df_compare["imc_survival"].isna() & df_compare["sma_survival"].notna()
]
print("IMC NULL, SMA tiene survival:", len(imc_null_sma_value))

sma_null_imc_value = df_compare[
    df_compare["sma_survival"].isna() & df_compare["imc_survival"].notna()
]
print("SMA NULL, IMC tiene survival:", len(sma_null_imc_value))


IMC NULL, SMA tiene survival: 137
SMA NULL, IMC tiene survival: 6


In [79]:
diff_survival = df_compare[df_compare["diff"].abs() > 0.00001]
diff_survival = df_compare[
    df_compare["imc_survival"].notna() &
    (df_compare["diff"].abs() > 0.00001)
]


In [80]:
print(len(diff_survival))  # deberías volver a ver tus 214

214


In [63]:
df_top_diffs = df_compare[
    df_compare["imc_survival"].notna() &
    (df_compare["diff"].abs() > 0.00001)
].sort_values(
    by="diff",
    key=abs,
    ascending=False
)

print(len(df_top_diffs))
df_top_diffs[[
    "contract_code",
    "sma_survival",
    "imc_survival",
    "diff",
    "inventory_year",
    "status",
    "planting_year",
    "region"
]].head()


214


,contract_code,sma_survival,imc_survival,diff,inventory_year,status,planting_year,region
68,CR0070,0.000000,1.0000,1.000000,2021.0,Out of Program,2021,Costa Rica
371,US0136,0.000000,0.9533,0.953300,2022.0,RFT,2022,USA
51,CR0053,0.953000,0.0000,-0.953000,2024.0,Active,2020,Costa Rica
152,MX0003,0.000000,0.9510,0.951000,2021.0,RFT,2021,Mexico
171,MX0022,0.983814,0.0541,-0.929714,2025.0,Active,2021,Mexico
